In [13]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.duet_neff import get_neff
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate




from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [7]:
siglimit = 10. # sigma
read_noise = 3. # Read noise for two frames

duet = config.Telescope()
duet.info()
print()
neff = get_neff(duet.psf_size, duet.pixel)
print('Number of effective bgd pixels: {}'.format(neff))



# point_drift = 1*u.arcsec # To be added to the PSF in quadrature
# point_jitter = 5*u.arcsec


# # Transmission efficiency
# trans_eff = (0.975)**8 # from Jim. 

# print('Transmission efficiency: {}'.format(trans_eff))

# # Load telescope info
# config=11
# diameter, qe, psf_size, pixel_size, efficiency = load_telescope_parameters(config)

# area = np.pi * (diameter*0.5)**2
# area_onaxis = np.pi * (0.5*24.2*u.cm)**2
# efficiency = area_onaxis / area

# print('Effective area (used) {}'.format(area*efficiency))

# # Add in fuzz...
# psf_size = (psf_size**2 + point_drift**2 + point_jitter**2)**0.5


# print('Effective PSF size {}'.format(psf_size))
# # Get the number of effective background pixels
# print()


Physical Entrance Pupil: 26.0 cm
Effective EPD:  24.2 cm
Effective Area:  4.6e+02 cm2
Pixel size:   6.4 arcsec
Transmission Efficiency:  0.82
PSF FWHM:   4.3 arcsec

Pointing jitter: 5.0 arcsec
Effective PSF FWHM: 6.6 arcsec

Band 1: {'eff_wave': <Quantity 213.60141673 nm>, 'eff_width': <Quantity 53.32814342 nm>}
Bandpass 1: [186.93734502 240.26548844] nm
Band 2: {'eff_wave': <Quantity 291.30506026 nm>, 'eff_width': <Quantity 68.16239088 nm>}
Bandpass 2: [257.22386482 325.3862557 ] nm

Number of effective bgd pixels: 2.922451027617772


In [9]:
# Band one
light=True

center = 208
width = 53
bandone=[center - 0.5*width, center+0.5*width]*u.nm
bandone = duet.bandpass1

print('For 10e3 K source')
for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, light=light)
    band1_rate = duet.trans_eff  * duet.eff_area * band1_fluence

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))


        break
print()
print('For 20e3 K source')


for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
    band1_rate = duet.trans_eff  * duet.eff_area * band1_fluence

    if (band1_rate.value < 1.0):
        bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandone = bandone)
        print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
        print('Band 1 Photometric Zero Point: {}'.format(bbmag1))
        break


For 10e3 K source
Band 1 Photometric Zero Point (Swift UVW2): 20.950000000000042
Band 1 Photometric Zero Point: 20.730689158259437 mag(AB)

For 20e3 K source
Band 1 Photometric Zero Point (Swift UVW2): 20.75000000000004
Band 1 Photometric Zero Point: 20.716078321602712 mag(AB)


In [12]:
light=True

center = 284
width = 68
bandtwo=[center - 0.5*width, center+0.5*width]*u.nm
bandtwo = duet.bandpass2

for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, light=light)
    band2_rate = duet.trans_eff  * duet.eff_area * band2_fluence

    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=10e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break
        
print()       
for swiftmag in np.arange(18, 25, 0.05):
    band1_fluence, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
    band2_rate = duet.trans_eff  * duet.eff_area * band2_fluence

    if (band2_rate.value < 1.0):
        foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandtwo = bandtwo)
        print('Band 2 Photometric Zero Point: {}'.format(swiftmag))
        print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
        break

Band 2 Photometric Zero Point: 21.750000000000053
Band 2 Photometric Zero Point: 20.5808999011315 mag(AB)

Band 2 Photometric Zero Point: 20.650000000000038
Band 2 Photometric Zero Point: 20.581594116006517 mag(AB)


In [15]:
light=True
siglimit=10


exp1 = []
sm = []

[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
# Compute sky background
# bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
#                               diameter=diameter,
#                               pixel_size=pixel_size, real_red=True, light=light)
#bgd_band1 = duet.* bgd_ph_rate
print('Sky background rate per pixel band1: {}'.format(bgd_band1))

# bgd_ph_rate = bgd_sky_qe_rate(low_zodi=True,
#                               diameter=diameter,
#                               qe_band=2,
#                               pixel_size=pixel_size, real_red=True, light=light)
#bgd_band2 = trans_eff  * efficiency * bgd_ph_rate
print('Sky background rate per pixel band2: {}'.format(bgd_band2))


bandone = duet.bandpass1
bandtwo = duet.bandpass2

# center = 208
# width = 53
# bandone=[center - 0.5*width, center+0.5*width]*u.nm

# center = 284
# width = 68
# bandtwo=[center - 0.5*width, center+0.5*width]*u.nm



print()
for siglimit in [5, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900]):
        for swiftmag in np.arange(20, 24, 0.05):
            band1_fluence, foo = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
            band1_rate = duet.trans_eff  * duet.eff_area * band1_fluence
            exposure1 = calc_exposure(siglimit, band1_rate, bgd_band1,read_noise*(nframes+1)**0.5, neff)
            if (exposure1.value > explimit):
                bbmag1, foo = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandone = bandone)
                print('Band 1 {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag1))
                break

print()

for siglimit in [5, 10]:
    print()
    for nframes, explimit in enumerate([300, 600, 900, 1200]):
        for swiftmag in np.arange(20, 24, 0.05):
            foo, band2_fluence = bb_abmag_fluence(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, light=light)
            band2_rate = duet.trans_eff * duet.eff_area * band2_fluence
            exposure2 = calc_exposure(siglimit, band2_rate, bgd_band2,read_noise*(nframes+1)**0.5, neff)
            if (exposure2.value > explimit):
                foo, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=20e3*u.K, bandtwo = bandtwo)
                print('Band 2 {}-s {}-σ magnitude limit: {}'.format(explimit, siglimit, bbmag2))
                break

Sky background rate per pixel band1: 0.030369732491096913 1 / s
Sky background rate per pixel band2: 0.24785885092656168 1 / s


Band 1 300-s 5-σ magnitude limit: 22.61607832160271 mag(AB)
Band 1 600-s 5-σ magnitude limit: 23.116078321602718 mag(AB)
Band 1 900-s 5-σ magnitude limit: 23.366078321602725 mag(AB)

Band 1 300-s 10-σ magnitude limit: 21.516078321602695 mag(AB)
Band 1 600-s 10-σ magnitude limit: 22.116078321602703 mag(AB)
Band 1 900-s 10-σ magnitude limit: 22.416078321602708 mag(AB)


Band 2 300-s 5-σ magnitude limit: 21.881594116006507 mag(AB)
Band 2 600-s 5-σ magnitude limit: 22.331594116006517 mag(AB)
Band 2 900-s 5-σ magnitude limit: 22.53159411600652 mag(AB)
Band 2 1200-s 5-σ magnitude limit: 22.731594116006523 mag(AB)

Band 2 300-s 10-σ magnitude limit: 20.981594116006494 mag(AB)
Band 2 600-s 10-σ magnitude limit: 21.4315941160065 mag(AB)
Band 2 900-s 10-σ magnitude limit: 21.681594116006504 mag(AB)
Band 2 1200-s 10-σ magnitude limit: 21.881594116006507 mag(AB)
